# Numerical Optmization Exam
Implementation of exercises from Nocedel-Wright.

In [21]:
import numpy as np

## Exercise 5.1

In [19]:
# Build the n×n Hilbert matrix  H_ij = 1/(i+j-1)
def hilbert(n):
    i = np.arange(1, n + 1)
    return 1.0 / (i[:, None] + i[None, :] - 1.0)

# Conjugate gradient method (Algorithm 5.2)
def cg(A, b, x0, tol, maxit, verbose = False):
    """Solve  A x = b  with CG.

    Parameters
    ----------
    A : SPD matrix (n×n)
    b : right‑hand side (n,)
    x0 : starting point (defaults to zeros)
    tol : stop when  ‖r_k‖ ≤ tol ‖r_0‖
    maxit : maximum iterations 
    
    Returns
    -------
    x      : approximate solution
    it     : iterations performed
    """
   
    n = b.size
    x = x0.copy()

    r = A @ x - b           # r0 = A x0 – b 
    p = -r.copy()           # p0 = -r0
    rs_old = r @ r          # r'r

    res0 = np.sqrt(rs_old)
    history: list[float] = [res0]

    for k in range(1, maxit + 1):
        alpha = rs_old / (p @ (A @ p))      # (5.24a)
        x += alpha * p                      # (5.24b)
        r += alpha * (A @ p)                # (5.24c)

        rs_new = r @ r
        res = np.sqrt(rs_new)
        history.append(res)
        
        if verbose:
            print(f"k {k} -- res = {res}")

        if res <= tol * res0:
            return (x, k) 

        beta = rs_new / rs_old              # (5.24d)
        p = -r + beta * p                   # (5.24e)
        
        rs_old = rs_new

    return (x, maxit)

In [20]:
dims = [5, 8, 12, 20]
tol = 1e-6
print("n   iterations ")
print("-   ---------- ")

for n in dims:
    H = hilbert(n)
    b = np.ones(n)
    x0 = np.zeros(n)

    _, it = cg(H, b, x0, tol=tol, maxit=100)
    print(f"{n:<3d} {it:^10d}")

n   iterations 
-   ---------- 
5       6     
8       19    
12      37    
20      68    


## Exercise 14.15

In [16]:
def mehrotra_lp(A, b, c, max_iter = 10, eta = 0.99, verbose = False):
    
    m, n = A.shape

    # Initialize with large positive values
    x   = np.ones(n)
    s   = np.ones(n)
    lam = np.zeros(m)

    e = np.ones(n)

    for k in range(max_iter):
        r_b = A @ x - b
        r_c = A.T @ lam + s - c
        mu   = (x @ s) / n

        if verbose:
            print(f"iter {k +1:2d}  ‖r_b‖={np.linalg.norm(r_b):.1e}  ‖r_c‖={np.linalg.norm(r_c):.1e}  μ={mu:.1e}")
            

        # Predictor step (affine‑scaling step)
        X = np.diag(x)
        S = np.diag(s)
        J = np.block([[np.zeros((n, n)),   A.T,               np.eye(n)],
                      [A,                  np.zeros((m, m)),  np.zeros((m, n))],
                      [S,                  np.zeros((n, m)),  X],])
        rhs_aff = np.concatenate([-r_c, 
                                  -r_b, 
                                  -X @ S @ e ])
        delta_aff = np.linalg.solve(J, rhs_aff)                                           # (14.30)
        dx_aff = delta_aff[:n]
        dl_aff = delta_aff[n:n + m]
        ds_aff = delta_aff[n + m:]

        # Centering step (use affine-scaling step to compute centering parameter)
        neg_dx = dx_aff < 0
        if np.any(neg_dx):
            alpha_aff_pri = min(1.0, np.min(-x[neg_dx] / dx_aff[neg_dx]))                 # (14.32a)

        neg_ds = ds_aff < 0
        if np.any(neg_ds):
            alpha_aff_dua = min(1.0, np.min(-s[neg_ds] / ds_aff[neg_ds]))                 # (14.32b)

        mu_aff = ((x + alpha_aff_pri * dx_aff) @ (s + alpha_aff_dua * ds_aff)) / n        # (14.33)
        sigma  = (mu_aff / mu) ** 3                                                       # (14.34)

        # Corrector step
        dX_aff = np.diag(dx_aff)     
        dS_aff = np.diag(ds_aff)     
        rhs_corr = np.concatenate([-r_c,
                                   -r_b,
                                   -X @ S @ e - dX_aff @ dS_aff @ e + sigma * mu * e,])    # (14.35)
        delta_corr = np.linalg.solve(J, rhs_corr)
        dx_corr = delta_corr[:n]
        dl_corr = delta_corr[n:n + m]
        ds_corr = delta_corr[n + m:]

        # Step lengths
        neg_dx_corr = dx_corr < 0
        if np.any(neg_dx_corr):
            alpha_pri = min(1.0, eta*np.min(-x[neg_dx_corr] / dx_corr[neg_dx_corr]))        # (14.38)

        neg_ds_corr = ds_corr < 0
        if np.any(neg_ds_corr):
            alpha_dua = min(1.0, eta*np.min(-s[neg_ds_corr] / ds_corr[neg_ds_corr]))        # (14.38)

        # Update 
        x   += alpha_pri * dx_corr
        lam += alpha_dua * dl_corr
        s   += alpha_dua * ds_corr

    return x, lam, s

In [17]:
m = 5
n = 5
l = m + n

rng = np.random.default_rng()         
A = rng.standard_normal((l, l))

x = np.zeros(l)
x[:m] = rng.random(m)               

s = np.zeros(l)
s[m:] = rng.random(n)                  

lam = rng.standard_normal(l)

c = A.T @ lam + s
b = A @ x

x_opt, lam_opt, s_opt = mehrotra_lp(A, b, c, verbose=True)
print("\nOptimal objective value:", c @ x_opt)
print("Distance to true x     :", np.linalg.norm(x_opt - x))

iter  1  ‖r_b‖=8.4e+00  ‖r_c‖=1.0e+01  μ=1.0e+00
iter  2  ‖r_b‖=8.4e-02  ‖r_c‖=9.5e-02  μ=5.7e-02
iter  3  ‖r_b‖=8.4e-04  ‖r_c‖=9.5e-04  μ=5.8e-04
iter  4  ‖r_b‖=8.4e-06  ‖r_c‖=9.5e-06  μ=5.8e-06
iter  5  ‖r_b‖=8.4e-08  ‖r_c‖=9.5e-08  μ=5.8e-08
iter  6  ‖r_b‖=8.4e-10  ‖r_c‖=9.5e-10  μ=5.8e-10
iter  7  ‖r_b‖=8.4e-12  ‖r_c‖=9.5e-12  μ=5.8e-12
iter  8  ‖r_b‖=8.4e-14  ‖r_c‖=9.6e-14  μ=5.8e-14
iter  9  ‖r_b‖=1.1e-15  ‖r_c‖=1.9e-15  μ=6.2e-16
iter 10  ‖r_b‖=3.6e-16  ‖r_c‖=1.9e-15  μ=1.6e-17

Optimal objective value: 1.3804872504489165
Distance to true x     : 2.2790900065266266e-16


## Exercise 17.3
We will implement the Newton method for the unconstrained optmization, assuming unit-length steps.

In [22]:
def newton_unconstrained(f, grad, hess, x0, tol, max_it=100):
    x = x0.copy()
    for _ in range(max_it):
        g = grad(x)
        if np.linalg.norm(g) < tol:
            break
        p = -np.linalg.solve(hess(x), g)
        x += p                     
    return x

def penalty_minimisation():
    x0 = np.array([2.0, 0.0])      
    history = []

    for k, mu in enumerate([1, 10, 100, 1000], start=1):
        def Q(x):
            g = x[0]**2 + x[1]**2 - 2
            return x.sum() + 0.5*mu*g**2

        def grad_Q(x):
            g = x[0]**2 + x[1]**2 - 2
            return np.array([
                1 + 2*mu*g*x[0],
                1 + 2*mu*g*x[1],
            ])

        def hess_Q(x):
            x1, x2 = x
            g = x1**2 + x2**2 - 2
            return np.array([
                [2*mu*(3*x1**2 + x2**2 - 2), 4*mu*x1*x2],
                [4*mu*x1*x2, 2*mu*(3*x2**2 + x1**2 - 2)],
            ])

        tau = 1.0 / mu
        x0  = newton_unconstrained(Q, grad_Q, hess_Q, x0, tol=tau)
        history.append(x0.copy())
        print(f"µ={mu:4d}   x≈{x0},   ‖∇Q‖={np.linalg.norm(grad_Q(x0)):.2e}")

    return np.vstack(history)

In [24]:
res = penalty_minimisation()
print("\nSequence of minimisers:\n", res)

µ=   1   x≈[-0.66504064 -1.47614495],   ‖∇Q‖=8.52e-01
µ=  10   x≈[-1.00937068 -1.0160636 ],   ‖∇Q‖=5.30e-02
µ= 100   x≈[-1.00085259 -1.00164937],   ‖∇Q‖=3.89e-03
µ=1000   x≈[-1.00011968 -1.0001304 ],   ‖∇Q‖=7.04e-04

Sequence of minimisers:
 [[-0.66504064 -1.47614495]
 [-1.00937068 -1.0160636 ]
 [-1.00085259 -1.00164937]
 [-1.00011968 -1.0001304 ]]
